# Optimization

Before we get started, professor Shi. strongly recommends that we should read the following reference books:
  *非线性最优化基础 Fukushimu 科学出版社*

## 1. Gradient Descent Method

Gradient descent is a iterative optimization algorithm for finding the minimum of a function. It's very effective to solving the convex optimization problems. The defination of convex function $f(\boldsymbol{w})$ is as follows:
$$
f(t\boldsymbol{w}_1 + (1-t)\boldsymbol{w}_2) \leq tf(\boldsymbol{w}_1) + (1-t)f(\boldsymbol{w}_2), \quad \forall \boldsymbol{w}_1, \boldsymbol{w}_2 \in \mathbb{R}^n,\forall t\in [0, 1] \tag{1.1}
$$
In order to implement the gradient descent method, we follow the following steps:
- Step 1. Initialize the parameters $\boldsymbol{w}_0\in W$ and set the learning rate $\eta_{t}$.
- Step 2. Gradient Descent: $\boldsymbol{w}_{t+1}' = \boldsymbol{w}_t - \eta_{t}\nabla f(\boldsymbol{w}_t)$
- Step 3. Projection $\boldsymbol{w}_{t+1}=P_{w}\boldsymbol{w}_{t+1}'$
- Repeat step 2. and step 3. for T steps
- Step 4. Return $\bar{\boldsymbol{w}}_{T}=\frac{1}{T} \sum_{t=1}^{T}\boldsymbol{w}_{t}$<br>
  
Step 3. is to guarantee that the parameters $\boldsymbol{w}$ is in the feasible set $W$. $P_{W}(\boldsymbol{z})= \argmin_{\boldsymbol{x}\in W}||\boldsymbol{x}-\boldsymbol{z}||$.
If we take a constant learning rate $\eta_{t}=\eta$, and the function $f(\boldsymbol{w})$ is l-Lipschitz continuous and is bounded, then we have the following theorem:
>**Theorem 1.1**. $f(\boldsymbol{w})-\min_{\boldsymbol{w}\in W}f(\boldsymbol{w})\leq O(\frac{1}{\sqrt{T}})$, where $\eta=\Gamma/(l/ \sqrt{T})$ and $\Gamma$ is the Diameter of $W$ and l is the upper bound of the length of gradient $||\triangledown f||\leq l$ .

## 2. Conjugate Gradient Method
Sometimes linear equations $Ax=b$ can be transform into the form of solving a minimum problem $x=\min_{x}f,f=\frac{1}{2}x^{T}Ax-b^{T}x$ <br>
It's easy to know that the gradient of $f$ is $\triangledown f=Ax-b$ and the Hessian matrix is $A$.<br> Here we suppose that $A$ is a symmetric positive definite matrix. Then we can use the conjugate gradient method to solve the linear equations $Ax=b$. <br>
Before we show the process, we first introduce the following definition:

>**definition2.1** The vectors $p_{0}, p_{1}, \cdots, p_{k}$ are called conjugate with respect to the symmetric positive definite matrix $A$, or A-conjugate. if $p_{i}^{T}Ap_{j}=<p_{i},Ap_{j}>\equiv<p_{i},p_{j}>_{A}=0$ for $i\neq j$.

In order to implement the conjugate gradient method, we follow the following steps:
- Step 1. Initialize the $x_{0}$ and the residual $r_{0}=b-Ax_{0}$, $p_{0}=r_{0}$, $k=0$.
- Step 2. Compute the step size $\alpha_{k}=\frac{< r_{k},p_{k}>}{<p_{k},p_{k}>_{A}}$. Update the variables as follows:
  $$ x_{k+1}=x_{k}+\alpha_{k}p_{k} $$   
  $$ r_{k+1}=r_{k}-\alpha_{k}Ap_{k}$$  
  $$ \textcolor{blue}{p_{k+1}=r_{k+1}-\sum_{l=0}^{k}\frac{<r_{k+1},p_{l}>_{A}}{<p_{l},p_{l}>_{A}}p_{l}\equiv r_{k+1}-\sum_{l=0}^{k}\beta_{l}p_{l}}$$ 
  or
  $$ \textcolor{red}{\beta_{k}=\frac{<r_{k+1},p_{k}>_{A}}{<p_{k},p_{k}>_{A}}} $$
  $$ \textcolor{red}{p_{k+1}=r_{k+1}-\beta_{k}p_{k}} $$
  $$ k \rightarrow k+1 $$
- Step 3. Repeat step 2. until $||r_{k}||<\epsilon$ .
The details can be found in the `CGM.solve()` function in the code.

#### some explainations

If we can find a set of A-conjugate vectors $p_{1},\dots,p_{n}$, then we can express the solution $x$ as a linear combination of $p_{1},\dots,p_{n}$, i.e., $$x=\sum_{i=1}^{n}\alpha_{i}p_{i}$$.<br>
Then we can get the following equation by substituting the equation into $Ax=b$:
$$
\sum_{i=1}^{n}\alpha_{i}Ap_{i}=b
$$
And we can get the coefficient $\alpha_{j}$ by ordinary inner product with $p_{j}$ on both sides of the equation:
$$
\alpha_{j}=\frac{<b,p_{j}>}{<p_{j},p_{j}>_{A}}
$$

The question is, how do we find the A-conjugate vectors $p_{1},\dots,p_{n}$? <br>
We can find the answer by extending the **Gram-Schmidt method**. Suppose we have found $p_{1},\dots,p_{k}$, and  $y_{k}$ which is not a linear combination of these vectors(guaranting $p_{k+1}\neq 0$), then we can find $p_{k+1}$ by the following equation:
$$
\textcolor{blue}{p_{k+1}=y_{k}-\sum_{i=1}^{k}\frac{<y_{k},p_{i}>_{A}}{<p_{i},p_{i}>_{A}}p_{i}}
$$
Take $y_{k}=r_{k}$ we have the standard conjugate gradient method to update $p_{k}$. <br>
It's still not optimal method, since it requires to memorize all the vectors $p_{1},\dots,p_{k}$, which is not efficient. <br>
We can find a better method by the following equation:
$$
\textcolor{red}{p_{k+1}={y_{k}-\beta_{k}p_{k}}}
$$
To make sure $p_{k+1}\perp^{A}p_{k}$, we perform A-inner product to both sides of the equation to determine $\beta_{k}$:
$$
\textcolor{red}{\beta_{k}=\frac{<y_{k},p_{k}>_{A}}{<p_{k},p_{k}>_{A}}}
$$
It's worth noting that the equation is different from the equation in the book, and the generated direction $p_{k+1}$ is not necessarily A-conjugate with $p_{j},j<k$, and this kind of conjugate gradient method can takes more steps then the standard conjugate gradient method to reach a higher accuracy. <br>

>**Theorem 2.1**. $r_{l}\perp span\{r_{0}, \dots,r_{l-1}\}$,$p_{l}\perp^{A}span\{p_{0},p_{1},\dots,p_{l-1}\}$ and $span\{p_{0},\dots,p_{l-1}\}=span\{ r_{0},\dots,r_{l-1}\}=span\{ r_{0},Ar_{0},A^{l-1}r_{0}\}$

proof: Mathematical Induction:<br>
1. base case: $l=1$  Obviously $span\{ p_{0}\}=span\{r_{0}\}$ because $p_{0}=r_{0}$<br>
   $r_{1}\perp span\{r_{0}\} \Leftrightarrow <r_{0}-\alpha_{0} Ap_{0},r_{0}>=0 $
   $  \Leftrightarrow \alpha_{0}=\frac{<r_{0},r_{0}>}{<p_{0},r_{0}>_{A}}$ By Definition $ \alpha_{0} =\frac{<b,p_{0}>}{<p_{0},p_{0}>_{A}}$ it holds$<br>
   $p_{1}\perp^{A}span\{p_{0}\} \Leftrightarrow <r_{1}-\beta_{0}p_{0},p_{0}>_{A}=0\Leftrightarrow \beta_{0}=\frac{<r_{1},p_{0}>_{A}}{<p_{0},p_{0}>_{A}}$ 
   It holds by the definition of $\beta_{0}$ <br>
2. Induction hypothesis: assume when $l=k$ the theorem holds<br> when it comes to $l=k+1$<br>
   First, $r_{k+1}\perp span\{r_{0},\dots,r_{k}\} \Leftrightarrow <r_{k}-\alpha_{k} Ap_{k},r_{j}>=0 , j=0,\dots,k $<br>
   for $ j<k $, from the induction hypothesis $ <r_{k},r_{j}>=0$, $<p_{k},r_{j}>_{A}=0$ $\therefore <r_{k}-\alpha_{k} Ap_{k},r_{j}>=<r_{k},r_{j}>-\alpha_{k} <p_{k},r_{j}>_{A}=0$
   for $j=k$,$<r_{k}-\alpha_{k} Ap_{k},r_{j}>=0 \Leftrightarrow \alpha_{k}=\frac{<r_{k},r_{k}>}{<p_{k},r_{k}>_{A}}$<br>
   $\because <r_{k},r_{k}>=<r_{k},p_{k}+\sum_{l=0}^{k-1}\beta_{l}p_{l}>=<r_{k},p_{k}>$ and $<p_{k},r_{k}>_{A}=<p_{k},r_{k}-\sum_{l=0}^{k-1}\beta_{l}p_{l}>_{A}=<p_{k},p_{k}>_{A}$<br>
   $\therefore \alpha_{k}=\frac{< r_{k},p_{k}>}{<p_{k},p_{k}>_{A}}=\frac{<r_{k},r_{k}>}{<p_{k},r_{k}>_{A}}$<br>

   Second, $p_{k+1}\perp^{A}span\{p_{0},p_{1},\dots,p_{k}\}$ is obvious by performing A-inner product with $p_{j},j=0,\dots,k$<br>
   
   In order to prove that $span\{p_{0},\dots,p_{k}\}=span\{ r_{0},\dots,r_{k}\}$, we only need to prove that $p_{k}$ can be expressed by linear combination of $r_{0},\dots,r_{k}$, because it's proved that vector systems that are perpendicular or A-conjugate to each other are linearly independent.<br>
   $\because p_{k}=r_{k}-\sum_{l=0}^{k-1}\beta_{l} p_{l}$ and from induction hypothesis $p_{l}$ can be expressed by linear combination of $r_{0},\dots,r_{l},l=0,\dots,k-1$<br>
   $\therefore p_{k}$ can be expressed by linear combination of $r_{0},\dots,r_{k}$<br>
   Finally, to prove that $span\{ r_{0},Ar_{0},A^{k}r_{0}\}= span\{r_{0}, \dots,r_{k}\}$ We denote that if a vector $x$ can be expressed by linear combination of $x_{0}, \dots,x_{k}$, then $x=L(x_{0}, \dots,x_{k})$<br>
   $$
   \begin{aligned}
   &r_{l+1}=r_{l}-\alpha_{l}Ap_{l}=r_{l}-\alpha_{l}A(r_{l}-\sum_{i=0}^{l-1}\beta_{i}p_{i})\\
   &\Rightarrow Ar_{l}=\frac{1}{\alpha_{l}}(r_{l}-r_{l+1})+AL(p_{0},\dots,p_{l-1})\\
   &=L(r_{l},r_{l+1})+L(Ap_{0},\dots,Ap_{l-1})\\
   &=L(r_{l},r_{l+1})+L(r_{1}-r_{0},\dots,r_{l}-r_{l-1})\\
   &=L(r_{0},\dots,r_{l+1})\\
   \end{aligned}
   $$
   We didn't use the induction hypothesis here, so it holds for all $l=0,\dots,k$ It's worth mentioning that the coefficient of $r_{k+1}\neq 0$ in the expression $Ar_{l}=L(r_{0},\dots,r_{l+1})$<br>
   Now,
   $$
   \begin{aligned}
   &A^{l}r_{0}=A^{l-1}L(r_{0},r_{1})\\
   &          =A^{l-2}L(Ar_{0},Ar_{1})\\
   &          =A^{l-2}L(L(r_{0},r_{1}),L(r_{0},r_{1},r_{2}))\\
   &          =A^{l-2}L(r_{0},r_{1},r_{2})\\
   &          =\dots\\
   &          =L(r_{0},\dots,r_{l}),l=0,\dots,k\\
   \end{aligned}
   $$
   Still, the coefficient of $r_{l}\neq 0$ in the expression.<br>
   So we can conclude that $A^{k}r_{0}$ can be linearly expressed by $ r_{0},r_{1},\dots,r_{k}$<br>
   And $A^{k}r_{0}$ is linearly independent with $r_{0},Ar_{0},\dots,A^{k-1}r_{0}$. Otherwise, we have:
   $$
   \begin{aligned}
   &A^{k}r_{0}=L(r_{0},\dots,A^{k-1}r_{0})\\
   &=L(r_{0},\dots,r_{k-1})\\
   \end{aligned}
   $$
   But $ A^{k}r_{0}=L(r_{0},\dots,r_{k})$ with nonzero coefficient of $r_{k}$, which is a contradiction.<br>
   Therefore, we can inductively prove that $r_{0},Ar_{0},\dots,A^{k}r_{0}$ are linear independent so that $span\{ r_{0},Ar_{0},A^{k}r_{0}\}= span\{r_{0}, \dots,r_{k}\}$<br>

In [263]:
class CGM():
    def __init__(self,A,b):
        self.A = A
        self.b = b
        self.x = np.zeros_like(b)
    def solve(self,A=None,b=None,Gram_Schmidt=False,record_list=False):
        if A is not None:
            self.A = A
        if b is not None:
            self.b = b
            self.x = np.zeros_like(b)
        r = self.b - np.dot(self.A,self.x)
        p = r
        steps = 0
        if Gram_Schmidt==True:
            p_list = []
            Ap_list = []
            p_list.append(p)
            while np.linalg.norm(r) > 1e-10 and steps<self.A.shape[0]:
                Ap = np.dot(self.A,p)
                Ap_list.append(Ap)
                alpha = np.dot(self.b,p)/np.dot(p,Ap)
                #alpha = np.dot(r,r)/np.dot(p,Ap) #way3 to calculate alpha
                self.x = self.x + alpha*p
                r_temp = r
                r = r - alpha*Ap
                p = r
                for i in range(len(p_list)):
                    p = p - np.dot(r,Ap_list[i])/np.dot(p_list[i],Ap_list[i])*p_list[i]
                #print(np.dot(p,Ap))
                p_list.append(p)
                steps+=1
            if record_list==True:
                self.p_list = p_list
                self.Ap_list = Ap_list
        else:
            if record_list==True:
                p_list = []
                Ap_list = []
                p_list.append(p)
            while np.linalg.norm(r) > 1e-10:
                Ap = np.dot(self.A,p)
                if record_list==True:
                    Ap_list.append(Ap)
                #alpha = np.dot(r,p)/np.dot(p,Ap)
                #alpha = np.dot(self.b,p)/np.dot(p,Ap) #way2 to calculate alpha
                alpha = np.dot(r,r)/np.dot(p,Ap) #way3 to calculate alpha
                self.x = self.x + alpha*p
                r_temp = r
                r = r - alpha*Ap   
                beta = np.dot(r,r)/np.dot(r_temp,r_temp)
                #beta = -np.dot(r,Ap)/np.dot(p,Ap)
                p = r + beta*p
                if record_list==True:
                    p_list.append(p)
                steps+=1
            if record_list==True:
                self.p_list = p_list
                self.Ap_list = Ap_list
        self.steps = steps

        return self.x

In [262]:
A = np.array([[2,1,0],
              [1,2,0.5],
              [0,0.5,1]])#A is symmetric and positive definite
b = np.array([3,4,1.0])
cgm=CGM(A,b) 

In [207]:
cgm.solve(Gram_Schmidt=True)

array([0.7, 1.6, 0.2])

In [208]:
np.dot(A,cgm.x)

array([3., 4., 1.])

In [276]:
#A100 = np.random.rand(100,100)
#A100 = np.dot(A100.T,A100)
#b100 = np.random.rand(100)
A100 = np.load('./data/A100.npy')
b100 = np.load('./data/b100.npy')
x100_np_linalg_solve=np.linalg.solve(A100,b100)

In [277]:
x100_cgm_solve=cgm.solve(A100,b100,Gram_Schmidt=False,record_list=True)
print(f'solution error by numpy: {np.linalg.norm(np.dot(A100,x100_np_linalg_solve)-b100)}')
print(f'solution error by CGM without Gram_Schmidt: {np.linalg.norm(np.dot(A100,x100_cgm_solve)-b100)}')
print(f'solution steps by CGM without Gram_Schmidt: {cgm.steps}')

solution error by numpy: 1.3394030069294292e-10
solution error by CGM without Gram_Schmidt: 1.245655570793503e-10
solution steps by CGM without Gram_Schmidt: 209


In [278]:
x100_cgm_solve=cgm.solve(A100,b100,Gram_Schmidt=True,record_list=True)
print(f'solution error by numpy: {np.linalg.norm(np.dot(A100,x100_np_linalg_solve)-b100)}')
print(f'solution error by CGM with Gram_Schmidt: {np.linalg.norm(np.dot(A100,x100_cgm_solve)-b100)}')
print(f'solution steps by CGM with Gram_Schmidt: {cgm.steps}')

solution error by numpy: 1.3394030069294292e-10
solution error by CGM with Gram_Schmidt: 6.484412297666219e-10
solution steps by CGM with Gram_Schmidt: 100


In [279]:
x100_cgm_solve=cgm.solve(A100,b100,Gram_Schmidt=False,record_list=True)
for i in range(10):
    print(np.dot(cgm.p_list[1],cgm.Ap_list[i]))

6.805852385788199e-13
97.15625118834603
6.718641725959403e-13
-2.40455594709508e-10
8.052655063280934e-08
-2.9214271034967522e-05
0.008658304451625676
-3.0363528805898143
371.7791040055364
-0.014809322889929663


From the result above, we can know that the conjugate gradient method without schmidt process can also converge to the solution of the linear equations $Ax=b$ with a higher accuracy. But the directions $p_{i}$ are not necessarily A-conjugate with each other. <br>